In [1]:
from scipy.optimize import fsolve
import numpy as np

In [2]:
def equations(p):
    x, y = p
    return (x**2 - y, np.exp(x) - y**2)

In [3]:
sol=fsolve(equations,(10,10))

In [4]:
equations(sol)

(0.0, 4.2655301513150334e-10)

In [5]:
sol

array([ 8.61316946, 74.18668809])